### PROGETTO FINALE IDS 


Utilizzeremo un dataset della stagione di calcio della Serie A 2023/2024, per provare a predire le ultime 5 giornate che vanno dalla 33° giornata alla 38°giornata.
In un campionato con 20 squadre e un totale di 38 giornate, ogni squadra gioca 38 partite (una per giornata).

Vogliamo considerare le ultime 5 giornate del campionato, quindi 50 partite in totale (10 partite per giornata × 5 giornate).

Per ciascuna partita, proveremo a prevedere il risultato usando uno dei seguenti segni:

1: vittoria della squadra di casa

X: pareggio

2: vittoria della squadra in trasferta

In [167]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

### PASSO 1 INTEGRIAMO IL FILE DELLE PARTITE E LO PULIAMO

Importiamo la tabella Serie A_matches_23_24.csv che contiene i risultati di tutte le partite del campionato e guardiamo cosa contiene con .head

In [168]:
matches = pd.read_csv('Serie A_matches_23_24.csv')
matches.head()


,Round,Round Name,Home Team,Away Team,UTC Time,Finished,Started,Cancelled,Awarded,Score,Match Status,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,1,Empoli,Hellas Verona,2023-08-19T16:30:00Z,True,True,False,False,0_1,Full-Time,NaN,NaN,NaN,NaN
1,1,1,Frosinone,Napoli,2023-08-19T16:30:00Z,True,True,False,False,1_3,Full-Time,NaN,NaN,NaN,NaN
2,1,1,Genoa,Fiorentina,2023-08-19T18:45:00Z,True,True,False,False,1_4,Full-Time,NaN,NaN,NaN,NaN
3,1,1,Inter,Monza,2023-08-19T18:45:00Z,True,True,False,False,2_0,Full-Time,NaN,NaN,NaN,NaN
4,1,1,Roma,Salernitana,2023-08-20T16:30:00Z,True,True,False,False,2_2,Full-Time,NaN,NaN,NaN,NaN


Come si puo notare dalla tabella, Round e roundname indicano la stessa cosa.
Ci sono delle colonne vuote e le eliminiamo.

In [169]:
drop_columns = ['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14','Round Name']
matches.drop(drop_columns, axis=1, inplace=True)


In [170]:
matches.head()

,Round,Home Team,Away Team,UTC Time,Finished,Started,Cancelled,Awarded,Score,Match Status
0,1,Empoli,Hellas Verona,2023-08-19T16:30:00Z,True,True,False,False,0_1,Full-Time
1,1,Frosinone,Napoli,2023-08-19T16:30:00Z,True,True,False,False,1_3,Full-Time
2,1,Genoa,Fiorentina,2023-08-19T18:45:00Z,True,True,False,False,1_4,Full-Time
3,1,Inter,Monza,2023-08-19T18:45:00Z,True,True,False,False,2_0,Full-Time
4,1,Roma,Salernitana,2023-08-20T16:30:00Z,True,True,False,False,2_2,Full-Time


In [171]:
#controlliamo se ci sono valori nulli
matches.isnull().sum()

Round           0
Home Team       0
Away Team       0
UTC Time        0
Finished        0
Started         0
Cancelled       0
Awarded         0
Score           0
Match Status    0
dtype: int64

Ci siamo accorti che la colonna Score che indica come è andato a finire il risultato di un match contiene il valore del risultato in una singola colonna, cerchiamo di dividere in 2 colonne separate questi valori salvandoli in Score_casa e Score_trasferta

In [172]:
matches[['Score_casa', 'Score_trasferta']] = matches['Score'].str.split('_', expand=True).astype(int)
matches.drop('Score', axis=1, inplace=True)
matches.head()

,Round,Home Team,Away Team,UTC Time,Finished,Started,Cancelled,Awarded,Match Status,Score_casa,Score_trasferta
0,1,Empoli,Hellas Verona,2023-08-19T16:30:00Z,True,True,False,False,Full-Time,0,1
1,1,Frosinone,Napoli,2023-08-19T16:30:00Z,True,True,False,False,Full-Time,1,3
2,1,Genoa,Fiorentina,2023-08-19T18:45:00Z,True,True,False,False,Full-Time,1,4
3,1,Inter,Monza,2023-08-19T18:45:00Z,True,True,False,False,Full-Time,2,0
4,1,Roma,Salernitana,2023-08-20T16:30:00Z,True,True,False,False,Full-Time,2,2


Ora creiamo una nuova colonna "Risultato" che va a confrontare i valori di score casa e score trasferta e li inserisce nella nuova colonna.

Se score_casa e > di score_trasferta allora risultato = 1.
Se score_casa e < di score_trasferta allora risultato = 2.
Se score_casa e = a score_trasferta allora risultato = X.

In [173]:
matches.loc[matches['Score_casa'] > matches['Score_trasferta'], 'Risultato'] = '1'
matches.loc[matches['Score_casa'] < matches['Score_trasferta'], 'Risultato'] = '2'
matches.loc[matches['Score_casa'] == matches['Score_trasferta'], 'Risultato'] = 'X'

# Eliminiamo le colonne non più utili dal dataframe
columns_to_drop = ["UTC Time", "Finished", "Started", "Cancelled", "Awarded", "Match Status"]
matches.drop(columns=columns_to_drop, inplace=True)

# Confermiamo le colonne rimanenti
matches.columns
matches.head()



,Round,Home Team,Away Team,Score_casa,Score_trasferta,Risultato
0,1,Empoli,Hellas Verona,0,1,2
1,1,Frosinone,Napoli,1,3,2
2,1,Genoa,Fiorentina,1,4,2
3,1,Inter,Monza,2,0,1
4,1,Roma,Salernitana,2,2,X


Ora che abbiamo la tabella pulita dividiamo in due la tabella matches.
train per le partite che vanno dalle 1 alla 32 e test per quelle dalla 33 alla 38
Così potremo lavorare solo sui matche che ci interessano per addestrare il modello e predire le ultime 5 giornate.

In [174]:
# Iniziamo isolando i dati di addestramento (giornate 1-32) e di test (giornate 33-38)
train_df = matches[matches["Round"] <= 32].copy()
test_df = matches[matches["Round"] >= 33].copy()

# Mostriamo un riepilogo delle dimensioni e le prime righe di ciascun set
train_df.head()




,Round,Home Team,Away Team,Score_casa,Score_trasferta,Risultato
0,1,Empoli,Hellas Verona,0,1,2
1,1,Frosinone,Napoli,1,3,2
2,1,Genoa,Fiorentina,1,4,2
3,1,Inter,Monza,2,0,1
4,1,Roma,Salernitana,2,2,X


Costruiamo ora i dati in tempo reale.
Per ogni squadra e per ogni giornata andremo ad aggiornare le statistiche in tempo reale.
avremo così una tabella che si aggiorna ad ogni giornata con le colonne interessate aggiornate: 
- Home_GoalFatti  
- Home_GoalSubiti
- Home_Pts	 
- Away_GoalFatti  
- Away_GoalSubiti 
- Away_Pts

In [175]:
# Lista di tutte le squadre
teams = pd.unique(train_df[['Home Team', 'Away Team']].values.ravel())

# Iniziamo costruendo un dizionario con le statistiche iniziali per ogni squadra
team_stats = {team: {
    'partite_giocate': 0,
    'gol_fatti': 0,
    'gol_subiti': 0,
    'vittorie': 0,
    'pareggi': 0,
    'sconfitte': 0,
    'punti': 0
} for team in teams}

# Creeremo una lista per memorizzare le feature per ogni riga
match_features = []

# Ordiniamo per giornata per mantenere la cronologia
train_df = train_df.sort_values(by="Round")

# Cicliamo sulle partite per generare statistiche aggiornate PRIMA di ogni match
for idx, row in train_df.iterrows():
    home = row['Home Team']
    away = row['Away Team']
    gh = row['Score_casa']
    ga = row['Score_trasferta']
    result = row['Risultato']
    
    # Recuperiamo le statistiche PRIMA della partita
    home_stats = team_stats[home].copy()
    away_stats = team_stats[away].copy()
    
    # Salviamo le feature
    match_features.append({
        'Round': row['Round'],
        'Home Team': home,
        'Away Team': away,
        'Home_GoalFatti': home_stats['gol_fatti'],
        'Home_GoalSubiti': home_stats['gol_subiti'],
        'Home_Pts': home_stats['punti'],
        'Away_GoalFatti': away_stats['gol_fatti'],
        'Away_GoalSubiti': away_stats['gol_subiti'],
        'Away_Pts': away_stats['punti'],
        'Risultato': result
    })
    
    # Dopo aver salvato, aggiorniamo le statistiche
    team_stats[home]['partite_giocate'] += 1
    team_stats[away]['partite_giocate'] += 1
    team_stats[home]['gol_fatti'] += gh
    team_stats[home]['gol_subiti'] += ga
    team_stats[away]['gol_fatti'] += ga
    team_stats[away]['gol_subiti'] += gh
    
    if result == '1':
        team_stats[home]['vittorie'] += 1
        team_stats[home]['punti'] += 3
        team_stats[away]['sconfitte'] += 1
    elif result == '2':
        team_stats[away]['vittorie'] += 1
        team_stats[away]['punti'] += 3
        team_stats[home]['sconfitte'] += 1
    elif result == 'X':
        team_stats[home]['pareggi'] += 1
        team_stats[away]['pareggi'] += 1
        team_stats[home]['punti'] += 1
        team_stats[away]['punti'] += 1

# Convertiamo la lista di dizionari in un DataFrame
train_features_df = pd.DataFrame(match_features)
train_features_df.head(10)


,Round,Home Team,Away Team,Home_GoalFatti,Home_GoalSubiti,Home_Pts,Away_GoalFatti,Away_GoalSubiti,Away_Pts,Risultato
0,1,Empoli,Hellas Verona,0,0,0,0,0,0,2
1,1,Frosinone,Napoli,0,0,0,0,0,0,2
2,1,Genoa,Fiorentina,0,0,0,0,0,0,2
3,1,Inter,Monza,0,0,0,0,0,0,1
4,1,Roma,Salernitana,0,0,0,0,0,0,X
5,1,Sassuolo,Atalanta,0,0,0,0,0,0,2
6,1,Lecce,Lazio,0,0,0,0,0,0,1
7,1,Udinese,Juventus,0,0,0,0,0,0,2
8,1,Torino,Cagliari,0,0,0,0,0,0,X
9,1,Bologna,Milan,0,0,0,0,0,0,2


In [176]:
train_features_df.tail(10)


,Round,Home Team,Away Team,Home_GoalFatti,Home_GoalSubiti,Home_Pts,Away_GoalFatti,Away_GoalSubiti,Away_Pts,Risultato
310,32,Lazio,Salernitana,37,34,46,25,64,15,1
311,32,Lecce,Empoli,26,48,29,25,47,28,1
312,32,Torino,Juventus,31,29,44,45,24,62,X
313,32,Bologna,Monza,45,25,58,34,41,42,X
314,32,Napoli,Frosinone,48,38,48,38,61,26,X
315,32,Sassuolo,Milan,36,59,25,60,34,68,X
316,32,Inter,Cagliari,75,15,82,32,52,30,X
317,32,Fiorentina,Genoa,45,37,46,34,38,38,X
318,32,Atalanta,Hellas Verona,57,37,50,28,42,27,X
319,32,Udinese,Roma,30,47,28,56,35,55,2


#fare grafico rendimento score casa e score trasferta 
